In [25]:
import numpy as np

# DIN EN ISO 6946, Tabelle 10
Dicke_Luftschicht = ()
R_Luftschicht = ()
L = 0

richtung_WS = 1 #Richtung des Wärmestroms, 0: Aufwärts, 1:Horizontal, 2:Abwärts
r = 4 # 精度,小数点位数
homogen = True # homogene Bauteile, True or False

# input 'L' into list_Lambda if it's Luftschicht(ruhend)
## Dicke [m] / Lambda [W/mK] / Baustoff
m1 = np.array([[0.0125,0.21,'Gipskarton'],
            [0.0002,0.2,'PE-Flie'],
            [0.0125,0.21,'Gipskarton'],
            [0.01,0.04,'Mineralwolle'],
            [0.08,0.04,'Mineralwolle'],
            [0.01,0.04,'Mineralwolle'],
            [0.015,0.36,'Gipsfaser'],
            [0.06,0.035,'Faserdaemmstoffe']])

m1_p = 94 # %

m2 = np.array([[0.0125,0.21,'Gipskarton'],
            [0.0002,0.2,'PE-Flie'],
            [0.0125,0.21,'Gipskarton'],
            [0.01,60,'Stahl'],
            [0.08,0.04,'Luftschicht'],
            [0.01,60,'Stahl'],
            [0.015,0.36,'Gipsfaser'],
            [0.06,0.035,'Faserdaemmstoffe']])

m2_p = 5 # %

m3 = np.array([[0.0125,0.21,'Gipskarton'],
            [0.0002,0.2,'PE-Flie'],
            [0.0125,0.21,'Gipskarton'],
            [0.01,60,'Stahl'],
            [0.08,60,'Stahl'],
            [0.01,60,'Stahl'],
            [0.015,0.36,'Gipsfaser'],
            [0.06,0.035,'Faserdaemmstoffe']])

m3_p = 1 # %

list_d = [0.015,0.365,0.02,0.14,0.01] # unit [m]
list_Lambda = [1,0.81,1,0.035,1] # unit [W/mK]


In [26]:
# DIN EN ISO 6946, Tabelle 9
if richtung_WS == 0:
    R_si = 0.1
    R_se = 0.04
elif richtung_WS == 1:
    R_si = 0.13
    R_se = 0.04
elif richtung_WS == 2:
    R_si = 0.17
    R_se = 0.04
else:
    print('ERROR: Richtung des Wärmestroms inkorrekt (0 ~ 2)')

In [27]:
# functions
## calculate Wärmewiderstand R with Dicke d and Lambda.
def dL2R(d,Lambda):
    if homogen == True:
        R = d / Lambda
        R = round(R,r)
        return R
    else:
        pass

## calculate U value for homogene Bauteile
def homogene_bauteile_U(list_R):
    R_T = R_si + R_se + sum(list_R)
    U = 1 / R_T
    return U

## calulate Wärmewiderstand R with d and Lambda for inhomogene Bauteile


## calculate U value for inhomogene Bauteile
#def inhomogene_bauteile_U(list_R):



In [38]:
# calculate R value
list_R = []
m1_R = np.array([])
if homogen == True:
    if len(list_d) != len(list_Lambda):
        print('ERROR: d and Lambda not match')
    else:
        for i in range (0,len(list_d)):
            list_R.append(dL2R(list_d[i],list_Lambda[i]))
else:
    if len(m1) == len(m2) == len(m3):
        pass
    else:
        print('ERROR: d and Lambda not match')
    

#print(list_R)
#print(R_si,R_se)
print(m1_R)

In [29]:
U_value = round(homogene_bauteile_U(list_R),r)
print(U_value)

0.2143
